In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f heise-articles-2020.db || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.db.gz && gunzip heise-articles-2020.db.gz")
    newsticker_db = 'heise-articles-2020.db'
else:
    newsticker_db = '../99_Common/heise-articles-2020.db'

In [ ]:
import sqlite3 
import pandas as pd

sql = sqlite3.connect(newsticker_db)
df = pd.read_sql("SELECT id, datePublished, title, commentCount FROM articles \
                    WHERE datePublished<'2021-01-01' ORDER BY datePublished", 
                 sql, index_col="id", parse_dates=["datePublished"])

In [ ]:
df["normalizedCommentCount"] = df["commentCount"].fillna(0).map(int)
df.loc[df["normalizedCommentCount"]>500, "normalizedCommentCount"] = 500

In [ ]:
df_success = df[df["normalizedCommentCount"]>50].copy()
df_success["success"] = 1

df_no_success = df[df["normalizedCommentCount"]<10].copy()
df_no_success["success"] = 0

In [ ]:
min_success = min(len(df_success), len(df_no_success))

In [ ]:
sdf = pd.concat([df_success.sample(min_success, random_state=42),
                 df_no_success.sample(min_success, random_state=42)])

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(min_df=2, ngram_range=(1,2))
tfidf_vectors = tfidf_vectorizer.fit_transform(sdf["title"])
tfidf_vectors

In [ ]:
from sklearn.model_selection import train_test_split
(X_train, X_test, y_train, y_test) = train_test_split(tfidf_vectors, sdf["success"].values, 
                                                      train_size=0.75, random_state=42,
                                                      stratify=sdf["success"].values)

In [ ]:
from sklearn.svm import SVC
svc = SVC(random_state=42)
svc.fit(X_train, y_train)

In [ ]:
import numpy as np
pred_test  = svc.predict(X_test)
np.unique(pred_test == y_test, return_counts=True)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, pred_test))